In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [32]:
from src.inference import get_feature_store

In [33]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*28)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
    read_options={"timeout":600},
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 09:47:14,243 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 09:47:14,252 INFO: Initializing external client
2025-03-04 09:47:14,252 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 09:47:15,567 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Fetching data from 2025-02-04 14:47:14.243816+00:00 to 2025-03-04 13:47:14.243816+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.30s) 


In [34]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 15:00:00+00:00,2,0
1,2025-02-04 16:00:00+00:00,2,0
2,2025-02-04 17:00:00+00:00,2,0
3,2025-02-04 18:00:00+00:00,2,0
4,2025-02-04 19:00:00+00:00,2,0
...,...,...,...
168416,2025-03-04 09:00:00+00:00,263,134
168417,2025-03-04 10:00:00+00:00,263,116
168418,2025-03-04 11:00:00+00:00,263,98
168419,2025-03-04 12:00:00+00:00,263,100


In [35]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168421 entries, 0 to 171432
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         168421 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  168421 non-null  int32                  
 2   rides               168421 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 3.9 MB


In [36]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [37]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168421 entries, 0 to 171432
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         168421 non-null  datetime64[us]
 1   pickup_location_id  168421 non-null  int32         
 2   rides               168421 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 3.9 MB


In [ ]:
from src.data_utils import transform_ts_data_info_features_and_target
features = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

Skipping location_id 79: Not enough data to create even one window.
Skipping location_id 87: Not enough data to create even one window.
Skipping location_id 70: Not enough data to create even one window.
Skipping location_id 143: Not enough data to create even one window.
Skipping location_id 96: Not enough data to create even one window.
Skipping location_id 234: Not enough data to create even one window.
Skipping location_id 21: Not enough data to create even one window.
Skipping location_id 150: Not enough data to create even one window.
Skipping location_id 132: Not enough data to create even one window.
Skipping location_id 251: Not enough data to create even one window.
Skipping location_id 61: Not enough data to create even one window.
Skipping location_id 62: Not enough data to create even one window.
Skipping location_id 75: Not enough data to create even one window.
Skipping location_id 249: Not enough data to create even one window.
Skipping location_id 91: Not enough data t

ValueError: No data could be transformed. Check if input DataFrame is empty or window size is too large.

In [40]:
features

(      rides_t-504  rides_t-503  rides_t-502  rides_t-501  rides_t-500  \
 0             170           82           74           41          110   
 1              17           18           68           44          109   
 2              64           86           62           11            8   
 3              85           78           56           22          221   
 4               7          111           76           47          238   
 ...           ...          ...          ...          ...          ...   
 2003            0            0            0            1            0   
 2004            0            0            0            0            0   
 2005            0            0            1            0            0   
 2006            0            0            0            0            1   
 2007            0            3            0            1            1   
 
       rides_t-499  rides_t-498  rides_t-497  rides_t-496  rides_t-495  ...  \
 0              13           39

In [41]:
features,targets = features

In [42]:
from src.inference import load_batch_of_features_from_store

In [43]:
current_date

Timestamp('2025-03-04 14:47:14.243816+0000', tz='Etc/UTC')

In [44]:
features=load_batch_of_features_from_store(current_date)
features

2025-03-04 09:47:53,346 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 09:47:53,351 INFO: Initializing external client
2025-03-04 09:47:53,351 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 09:47:53,902 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Fetching data from 2025-02-03 14:47:14.243816+00:00 to 2025-03-04 13:47:14.243816+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.74s) 


(     rides_t-648  rides_t-647  rides_t-646  rides_t-645  rides_t-644  \
 0              0            0            0            0            0   
 1              0            0            0            0            0   
 2              1            0            1            0            0   
 3              0            0            1            0            0   
 4              1            1            2            1           11   
 ..           ...          ...          ...          ...          ...   
 497            1            0            1            7            8   
 498            3           19           63          126          177   
 499            5            6           13           66          151   
 500            7           21           46          142          111   
 501            4            4           17           48          136   
 
      rides_t-643  rides_t-642  rides_t-641  rides_t-640  rides_t-639  ...  \
 0              0            0            0 

In [45]:
from src.inference import load_model_from_registry
model = load_model_from_registry()

2025-03-04 09:48:09,630 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 09:48:09,630 INFO: Initializing external client
2025-03-04 09:48:09,630 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 09:48:10,219 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673


In [46]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

AttributeError: 'tuple' object has no attribute 'columns'

In [ ]:
predictions

In [ ]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values